# Custom Layers

One factor behind deep learning's success
is the availability of a wide range of layers
that can be composed in creative ways
to design architectures suitable
for a wide variety of tasks.
For instance, researchers have invented layers
specifically for handling images, text,
looping over sequential data,
and
performing dynamic programming.
Sooner or later, you will encounter or invent
a layer that does not exist yet in the deep learning framework.
In these cases, you must build a custom layer.
In this section, we show you how.

## (**Layers without Parameters**)

To start, we construct a custom layer
that does not have any parameters of its own.
This should look familiar if you recall our
introduction to block in :numref:`sec_model_construction`.
The following `CenteredLayer` class simply
subtracts the mean from its input.
To build it, we simply need to inherit
from the base layer class and implement the forward propagation function.


In [1]:
import torch
from torch import nn
from torch.nn import functional as F

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

Let us verify that our layer works as intended by feeding some data through it.


In [2]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

We can now [**incorporate our layer as a component
in constructing more complex models.**]


In [3]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

As an extra sanity check, we can send random data
through the network and check that the mean is in fact 0.
Because we are dealing with floating point numbers,
we may still see a very small nonzero number
due to quantization.


In [4]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(5.5879e-09, grad_fn=<MeanBackward0>)

In [5]:
Y.shape

torch.Size([4, 128])

## [**Layers with Parameters**]

Now that we know how to define simple layers,
let us move on to defining layers with parameters
that can be adjusted through training.
We can use built-in functions to create parameters, which
provide some basic housekeeping functionality.
In particular, they govern access, initialization,
sharing, saving, and loading model parameters.
This way, among other benefits, we will not need to write
custom serialization routines for every custom layer.

Now let us implement our own version of the  fully-connected layer.
Recall that this layer requires two parameters,
one to represent the weight and the other for the bias.
In this implementation, we bake in the ReLU activation as a default.
This layer requires to input arguments: `in_units` and `units`, which
denote the number of inputs and outputs, respectively.


In [6]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

Next, we instantiate the `MyLinear` class
and access its model parameters.


In [7]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-0.1008,  0.6590,  0.0711],
        [-1.9966,  0.0492,  1.2266],
        [ 2.2095,  0.2637,  1.3558],
        [ 0.0359, -1.8069, -0.3349],
        [-1.0383, -0.2622,  1.6580]], requires_grad=True)

We can [**directly carry out forward propagation calculations using custom layers.**]


In [8]:
linear(torch.rand(2, 5))

tensor([[0.0000, 0.7650, 2.9766],
        [0.0000, 1.1556, 2.8960]])

We can also (**construct models using custom layers.**)
Once we have that we can use it just like the built-in fully-connected layer.


In [9]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[ 8.1221],
        [14.9005]])

## Summary

* We can design custom layers via the basic layer class. This allows us to define flexible new layers that behave differently from any existing layers in the library.
* Once defined, custom layers can be invoked in arbitrary contexts and architectures.
* Layers can have local parameters, which can be created through built-in functions.


## Exercises

1. Design a layer that takes an input and computes a tensor reduction,
   i.e., it returns $y_k = \sum_{i, j} W_{ijk} x_i x_j$.
1. Design a layer that returns the leading half of the Fourier coefficients of the data.


[Discussions](https://discuss.d2l.ai/t/59)


### --- my exp ---

### c.f. The concept of blocks

In [10]:
class MLP(nn.Module):
    # Declare a layer with model parameters. Here, we declare two fully
    # connected layers
    def __init__(self):
        # Call the constructor of the `MLP` parent class `Module` to perform
        # the necessary initialization. In this way, other function arguments
        # can also be specified during class instantiation, such as the model
        # parameters, `params` (to be described later)
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # Hidden layer
        self.out = nn.Linear(256, 10)  # Output layer

    # Define the forward propagation of the model, that is, how to return the
    # required model output based on the input `X`
    def forward(self, X):
        # Note here we use the funtional version of ReLU defined in the
        # nn.functional module.
        return self.out(F.relu(self.hidden(X)))

In [11]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [12]:
net = MLP()
X = torch.rand(2, 20)
net(X)

tensor([[ 0.0409,  0.1636,  0.0674,  0.0077, -0.0920, -0.1379, -0.0128,  0.0105,
          0.1761, -0.1201],
        [ 0.1966,  0.1812, -0.0632, -0.1789, -0.1717, -0.0792,  0.1898, -0.0008,
          0.3511,  0.0458]], grad_fn=<AddmmBackward>)

In [13]:
linear = MyLinear(5, 3)

In [14]:
linear.weight.data

tensor([[ 0.9377,  1.3458, -0.2566],
        [-0.3703,  0.4770,  0.8712],
        [-1.9412,  1.5163, -1.2186],
        [-0.7162, -0.1622,  1.5379],
        [-0.3882, -1.1413,  0.8755]])

In [15]:
linearDef = nn.Linear(5, 3)
linearDef.weight.data

tensor([[-1.8588e-01, -4.4696e-01,  1.8048e-01, -3.4577e-03, -3.8395e-01],
        [-1.6218e-01, -1.0383e-01,  9.9868e-05, -6.2162e-02, -3.3762e-01],
        [ 8.4751e-02,  3.1901e-01,  2.5271e-01,  2.0767e-01, -8.1936e-02]])

In [16]:
net.hidden.weight.data

tensor([[-0.0948, -0.1355, -0.1927,  ..., -0.0276,  0.1660, -0.1413],
        [-0.2095,  0.1971,  0.1834,  ...,  0.2100,  0.0512,  0.1090],
        [ 0.0099,  0.1345,  0.1443,  ..., -0.1541,  0.1348,  0.1004],
        ...,
        [ 0.0707,  0.1510, -0.0998,  ..., -0.1880,  0.0483,  0.1392],
        [-0.1030, -0.0067, -0.1893,  ...,  0.0607, -0.0521, -0.0903],
        [-0.1383, -0.0842, -0.0954,  ...,  0.1150, -0.0419,  0.0630]])

In [17]:
nn.Linear

torch.nn.modules.linear.Linear